In [ ]:
import sys as _sys
import os

currentPath = os.path.abspath(os.getcwd())

split = currentPath.split("Cshells")
if len(split)<2:
    print("Please rename the repository 'Cshells'")
    raise ValueError
pathToPythonScripts = os.path.join(split[0], "Cshells/python/")
pathToModels = os.path.join(split[0], "Cshells/data/models")
pathToOutputs = os.path.join(split[0], "Cshells/output")

_sys.path.insert(0, pathToPythonScripts)

In [ ]:
import MeshFEM
import ElasticRods

import average_angle_linkages
import elastic_rods
import json
import math
import numpy as np
import pickle
import torch

from CShell import CShell, GetEdgesFromCurves
from CurvesDoFOptimizer import CurvesDoFOptimizer
from geomdl import BSpline, knotvector, tessellate
from ParametricFamily import GaussianRBF
from PlanarizationOptimizers import PlanarizationOptimizer, ComputeJointAngles
from LinkageTopologies import RegularTopology
from open_average_angle_linkage import open_average_angle_linkage
from tri_mesh_viewer import TriMeshViewer
from VisUtils import ConvergencePlotsVisualizer, PlotStackedConvergencePlots
from VisUtilsInitialization import PlotFlatLayout, PlotUV, PlotDeployedLayout

torch.set_default_dtype(torch.float64)
    
def ToNumpy(tensor):
    return tensor.cpu().detach().clone().numpy()

PI = math.pi

# Global Parameters

In [ ]:
model_name = "double_bumps"

pathToJson = os.path.join(pathToOutputs, "{}/planarization".format(model_name))
if not os.path.exists(pathToJson):
    os.makedirs(pathToJson)

matParams = [1.2, 0.2]
youngModulus = 2100.0
poissonRatio = 0.35
stiffAxis = (elastic_rods.StiffAxis.D1 if matParams[0] > matParams[1] else elastic_rods.StiffAxis.D2)

rodMaterial = elastic_rods.RodMaterial(
    'rectangle', youngModulus, poissonRatio,
    matParams, stiffAxis=stiffAxis, keepCrossSectionMesh=True
)
rodMaterialTwist = elastic_rods.RodMaterial(
    'rectangle', youngModulus, poissonRatio,
    matParams, stiffAxis=stiffAxis, keepCrossSectionMesh=True
)
rodMaterialTwist.twistingStiffness = 10.0 * rodMaterialTwist.twistingStiffness

# Define a target surface

The target surface is constructed using a heightfield $z:\mathbb{R}^2\rightarrow \mathbb{R}$ given a bimodal Gaussian radial basis function (see `GaussianRBF` in `ParametricFamily.py`). These are defined as

$$
z(\mathbf{x})= m + (M-m) \sum_i w_i \mathrm{exp}(-\epsilon^2||\mathbf{x} - \mathbf{c}_i||^2),
$$

where $\mathbf{c}_i\in\mathbb{R}^2$ are the centers of the RBF, $\mathbf{w}_i$ are the weigths for each RBF, $\epsilon$ is the extent of the RBFs, and $m$ and $M$ control the range of the heightfield.

In [ ]:
nX = 10
nY = 7

xMin = 0.0
xMax = 40.0
yMin = 0.0
yMax = 20.0
surfHeight = 10.0

xs = np.repeat(np.linspace(xMin, xMax, nX), (nY))
ys = np.tile(np.linspace(yMin, yMax, nY), (nX))

centers = np.array([
    xMin + (xMax - xMin) / 6, yMin + 0.2 * (yMax - yMin),
    xMin + 0.7 * (xMax - xMin), yMin + 0.6 * (yMax - yMin),
])
w1 = 0.43
weights = np.array([w1, 1.0 - w1])
params = np.concatenate([centers, weights], axis=0)
eps = 1 / (8.0)
rbf = GaussianRBF(params, eps, 0.0, surfHeight)
locs = np.stack([xs, ys], axis=1)
zs = rbf.eval(locs)

minZs = np.min(zs)
maxZs = np.max(zs)
zs -= minZs
zs *= surfHeight / (maxZs - minZs)

rbf.plot(xLimPlot=[xMin, xMax], yLimPlot=[yMin, yMax])

In [ ]:
# We transform the rectangular patch into a patch with boundaries on arc of circles
innerRad = 30.0
outerRad = 50.0
thetasSpan = np.pi / 3

rads = np.tile(np.linspace(innerRad, outerRad, nY), (nX))
thetas = np.repeat(np.linspace(-thetasSpan/2, thetasSpan/2, nX), (nY))

xsCP = rads * np.sin(thetas)
ysCP = rads * np.cos(thetas)

controlPoints = np.stack([xsCP, ysCP, zs], axis=1)
com = np.mean(controlPoints, axis=0, keepdims=True)
com[0, 2] = 0
controlPoints -= com

# Create a BSpline surface instance (Bezier surface)
surf = BSpline.Surface()

surf.degree_u = 3
surf.degree_v = 3
surf.set_ctrlpts(controlPoints.tolist(), nX, nY)
surf.knotvector_u = knotvector.generate(surf.degree_u, nX)
surf.knotvector_v = knotvector.generate(surf.degree_v, nY)

surfFaces = np.array([f.vertex_ids for f in surf.faces])
surfVerts = np.array([list(v.data) for v in surf.vertices])

view = TriMeshViewer((surfVerts, surfFaces), wireframe=True, transparent=False)
view.show()

# Initialize planarization

We use a "square" layout to initialize the linkage topology. The flat (`jointsFlatInit`) and target (`jointsDepInit`) linkages are rescaled so that the distance separating pairs of joints are comparable. Furthermore, the flat linkage is deformed according to the compatible scissor mechanism allowed by the joints positions.

In [ ]:
nJa  = 8 # Number of joints for the first family of joints
nJb  = 5  # Number of joints for the second family of joints

nJ, curves, curvesFamily = RegularTopology(nJa, nJb)
rodEdges, rodEdgeToCurve = GetEdgesFromCurves(curves)

uDepInit  = np.tile(np.linspace(0.0, 1.0, nJa), (nJb,))
vDepInit  = np.repeat(np.linspace(0.0, 1.0, nJb), nJa)
uvDepInit = np.stack([uDepInit, vDepInit], axis=1)
jointsDepInit = np.array(surf.evaluate_list(uvDepInit.tolist()))

# We compute the scale of the linkage on the target surface and scale the flat linkage accordingly
depLengthCurves = np.array([sum([np.linalg.norm(jointsDepInit[crv[i+1]] - jointsDepInit[crv[i]]) for i in range(len(crv)-1)]) for crv in curves])
scaleLengthA = np.mean(depLengthCurves[[fam == 0 for fam in curvesFamily]])
scaleLengthB = np.mean(depLengthCurves[[fam == 1 for fam in curvesFamily]])

# We simulate the compatible scissor mechanism by scaling the x and y directions
initShearingFactor = 1.5
scalingFactor = 1.0

xRange = scalingFactor * scaleLengthA
yRange = scalingFactor * scaleLengthB / np.sqrt(initShearingFactor ** 2 + 1)

xPos = np.tile(np.linspace(0.0, xRange, nJa), (nJb,))
yPos = np.repeat(np.linspace(0.0, yRange, nJb), nJa)
initShearing = np.array([[1.0, initShearingFactor], [0.0, 1.0]])
jointsFlatInit = np.stack([xPos, yPos], axis=1) @ initShearing.T
jointsFlatInit -= np.mean(jointsFlatInit, axis=0, keepdims=True)


In [ ]:
jointMarkerParams = {"s": 180, "lw": 4.0}
curveWidthParams = {"lwInner": 10.0, "lwOuter": 9.0}
PlotUV(
    uvDepInit, curves, 
    pathToSave=os.path.join(pathToJson, "diagram_uv_square.png"),
    jointMarkerParams=jointMarkerParams, curveWidthParams=curveWidthParams
)

jointMarkerParams = {"s": 200, "lw": 4.0}
curveWidthParams = {"lwInner": 10.0, "lwOuter": 9.0}
PlotFlatLayout(
    jointsFlatInit, curves, 
    pathToSave=os.path.join(pathToJson, "diagram_init_layout_square.png"),
    jointMarkerParams=jointMarkerParams, curveWidthParams=curveWidthParams
)

jointMarkerParams = {"s": 100}
curveWidthParams = {"lw": 7.0}
PlotDeployedLayout(
    jointsDepInit, curves, surf,
    pathToSave=os.path.join(pathToJson, "diagram_init_target_square.png"),
    jointMarkerParams=jointMarkerParams, curveWidthParams=curveWidthParams,
    azim=-125.0, elev=30.0,
)

# Planarization

The joint positions are optimized both in the flat and target layouts.

In [ ]:
xFlatInitPlan = torch.tensor(jointsFlatInit.reshape(-1,))
xDepInitPlan  = uvDepInit.copy().reshape(-1,)
xInitPlan = np.concatenate([xFlatInitPlan, xDepInitPlan], axis=0)

boundaryIndices = np.unique([crv[0] for crv in curves] + [crv[-1] for crv in curves])
pinnedJoint = np.argsort(jointsDepInit[:, 2])[-1]
pinnedDoFs = [2 * pinnedJoint, 2 * pinnedJoint + 1]

weightsPlan = {
    'length': 5.0e2,
    'angles': 1.0e1,
    'weightMeanAngle': 0.0,
    'eqSL': 8.0e-3,
    'bnd': 1.0e1,
    'flatReg': 1.0e-5,
    'areaConstraint': 0.02,
    'lenConstraint': 2.0,
    'bndOnly': 1,
}

plan_opt = PlanarizationOptimizer(
    curves, curvesFamily, surf, jointsFlatInit, jointsDepInit,
    "knitro_lbfgs.opt", pinnedVarsDepIdx=pinnedDoFs, rodMaterial=rodMaterialTwist,
    weightLengths=weightsPlan['length'], weightAngles=weightsPlan['angles'], 
    weightMeanAngle=weightsPlan['weightMeanAngle'], weightEqSL=weightsPlan['eqSL'],
    weightBnd=weightsPlan['bnd'], weightFlatReg=weightsPlan['flatReg'], 
    factorAreaConstraint=weightsPlan['areaConstraint'], factorLenConstraint=weightsPlan['lenConstraint'],
    bndOnly=weightsPlan['bndOnly'],
)

xOptPlan = plan_opt.Optimize(
    xInitPlan, numSteps=1000, trustRegionScale=1.0e-1,
    optTol=1.0e-5, ftol=1.0e-7, xtol=1.0e-6, 
    ftol_iters=5, xtol_iters=5, honorbounds=1,
    verbosity=2
)

valsFlatOptPlan  = xOptPlan[:plan_opt.nVarsFlat]
jointsFlatOptPlan = valsFlatOptPlan.copy().reshape(-1, 2)

valsDepOptPlan   = xOptPlan[plan_opt.nVarsFlat:].reshape(-1, 2)
jointsDepOptPlan = np.array(surf.evaluate_list(valsDepOptPlan.tolist().copy()))

jointsFlat3DOptPlan        = torch.zeros(size=(jointsFlatOptPlan.shape[0], 3))
jointsFlat3DOptPlan[:, :2] = torch.tensor(jointsFlatOptPlan)

In [ ]:
jointMarkerParams = {"s": 200, "lw": 4.0}
curveWidthParams = {"lwInner": 10.0, "lwOuter": 9.0}
PlotFlatLayout(
    jointsFlatOptPlan, curves, 
    pathToSave=os.path.join(pathToJson, "diagram_plan_layout_square.png"),
    jointMarkerParams=jointMarkerParams, curveWidthParams=curveWidthParams
)

jointMarkerParams = {"s": 100}
curveWidthParams = {"lw": 7.0}
PlotDeployedLayout(
    jointsDepOptPlan, curves, surf,
    pathToSave=os.path.join(pathToJson, "diagram_plan_target_square.png"),
    jointMarkerParams=jointMarkerParams, curveWidthParams=curveWidthParams,
    azim=-45.0, elev=20.0,
)

# Design Optimization

After initializing the linkage, we run design optimization to ameliorate the design in terms of target fitting and deployed elastic energy.

## Create the target mesh

We discretize the target surface so that it can be used during design optimizaiton

In [ ]:
surf.tessellator = tessellate.TriangularTessellate()
surf.tessellate()

targetMesh = {
    'V'        : np.array(surf.vertices),
    'F'        : np.array([list(face.vertex_ids) for face in surf.faces]),
    'targetJP' : jointsDepInit.reshape(-1,), # jointsDepOptSS, jointsDepInit
}

attractionMesh = targetMesh

## Initialize the linkage

We now deploy the linkage, initialized as an `AverageAngleSurfaceAttractedLinkage`, using the target mesh as the attraction surface.

In [ ]:
nCPperRodEdge = [(len(crv) - 1) * [1] for crv in curves]
totNCP = sum([sum(el) for el in nCPperRodEdge])
curvesDoF = torch.zeros(size=(2 * nJ + totNCP,))
curvesDoF[:2*nJ]  = torch.tensor(jointsFlatOptPlan.reshape(-1,))

anglesFlat  = ComputeJointAngles(jointsFlat3DOptPlan, plan_opt.idxEdgesA, plan_opt.idxEdgesB)
anglesDep   = ComputeJointAngles(torch.tensor(jointsDepOptPlan), plan_opt.idxEdgesA, plan_opt.idxEdgesB)
alphaTar    = torch.mean(anglesDep).item() - 0.2
alphaTar    = torch.mean(anglesDep).item() + 0.1

mult = 5
subdivision = 10

cshell = CShell(
    curvesDoF, nJ, curves, curvesFamily, nCPperRodEdge, alphaTar, mult, subdivision, 
    rodMaterial=rodMaterial, pathSurf=None, newtonOptimizerOptions=None, optimizeAlpha=True, 
    useSAL=True, attractionMesh=attractionMesh, attractionWeight=1.0e-3, targetMesh=targetMesh,
    dictWeights=None, linkagesGuess=None,
    numOpeningSteps=40, maxNewtonIterIntermediate=20, flatOnly=False
)

# Pop the mesh in the right configuration
cshell.UpdateAttractionWeights(1.0, cshell.attractionJointPosWeight)
cshell.UpdateAttractionWeights(1.0e-3, cshell.attractionJointPosWeight)

In [ ]:
cshell.flatView.show()

In [ ]:
cshell.deployedView.show()

In [ ]:
if not os.path.exists(pathToJson):
    os.makedirs(pathToJson)

with open(os.path.join(pathToJson, "cshell_initial_square.p"), 'wb') as f:
    pickle.dump(cshell.GetCShellParams(), f)

with open(os.path.join(pathToJson, "flat_initial_square.p"), 'wb') as f:
    pickle.dump(cshell.flatLinkage, f)
    
with open(os.path.join(pathToJson, "deployed_initial_square.p"), 'wb') as f:
    pickle.dump(cshell.deployedLinkage, f)


## Optimize the structure

Now that the C-shell is deployed, we can optimize the flat layout.

In [ ]:
minRL = None
swapAxesCS = False

dictWeights = {
    'beta':6.0e4,
    'gamma':0.0,
    'smoothingWeight':1.5e-3,
    'rlRegWeight':0.0,
    'cpRegWeight':3.0e2,
    'minAngle': np.pi / 64,
    'minArea': 0.1,
    'jointPosWeight': 0.1,
    'featureMultiplier': 200.0
}

pinnedJoint = np.argsort(jointsDepInit[:, 2])[-1]
featJoints = [pinnedJoint] + torch.argwhere(cshell.valence <= 3).reshape(-1,).tolist()


In [ ]:
screenshot = None
pathToSaveJSON = None

cshell.linkageOptimizer.scaleFeatureJointWeights(
    jointPosWeight=dictWeights['jointPosWeight'], 
    featureMultiplier=dictWeights['featureMultiplier'],
    feature_pts=featJoints
)
cshell.linkageOptimizer.setHoldClosestPointsFixed(False)
cshell.SetWeights(
    beta=dictWeights['beta'], 
    gamma=dictWeights['gamma'], 
    smoothingWeight=dictWeights['smoothingWeight'], 
    rlRegWeight=dictWeights['rlRegWeight'], 
    cpRegWeight=dictWeights['cpRegWeight'],
)

cdo = CurvesDoFOptimizer(cshell, "knitro_lbfgs.opt", minAngle=dictWeights['minAngle'], minRL=minRL, minArea=dictWeights['minArea'])

optDoF, optAlpha = cdo.OptimizeDoF(
    numSteps=10000, useCB=True, maxEqSteps=50, computeGradMags=False,
    ftol=1.0e-9, ftol_iters=3,
    screenshot=screenshot, visDeviations=True,
    saveGeometryPath=pathToSaveJSON, saveGeometryFrequency=3
)
nIterations = len(cshell.optimizationCallback.iterateData)

In [ ]:
with open(os.path.join(pathToJson, "flat_optimized_square.p"), 'wb') as f:
    pickle.dump(cshell.flatLinkage, f)

with open(os.path.join(pathToJson, "deployed_optimized_square.p"), 'wb') as f:
    pickle.dump(cshell.deployedLinkage, f)

with open(os.path.join(pathToJson, "cshell_optimized_square.p"), 'wb') as f:
    pickle.dump(cshell.GetCShellParams(), f)